In [6]:
search_for = 106
start_from = 0
threads = 1
thread = 0
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.016708850860595703
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 25
all cases: 249308851
type: [1, 1, 1, 1, 106] 106
cases of this type: 126247696
10000 37.93607258767394
20000 38.35270812927479
30000 40.97984371471802
40000 133.6362741365913
50000 223.19178336310918
60000 178.4750530117961
70000 165.74251084427425
80000 168.75987725789201
90000 180.93019162389402
100000 182.6866632813996
110000 169.35185558923172
120000 168.62879273460945
130000 176.32589528577876
140000 178.2840085810345
150000 173.8263082193852
160000 163.20919370905395
170000 172.95875378434172
180000 169.19502889029027
190000 170.97568708425672
200000 169.1467246295983
210000 161.1476767429626
220000 165.88870470204932
230000 173.98211620404354
240000 162.41968151548303
250000 172.41851772821423
260000 172.97146559341653
270000 162.9912342679275
280000 168.54842271275012
290000 170.7789840184103
300000 191.10693885684117
310000 185.68

3050000 174.65038260214246
3060000 160.6848445977147
3070000 152.43321309532053
3080000 152.56889433804477
3090000 164.10055737629958
3100000 166.0252634715104
3110000 158.1554922609957
3120000 153.65034799099715
3130000 161.02827351351772
3140000 159.79293699491205
3150000 157.7956240469015
3160000 152.9756590479385
3170000 154.87054750135744
3180000 153.12505732652974
3190000 156.2489385393979
3200000 152.7911156438804
3210000 159.17037914849837
3220000 157.4146891379482
3230000 163.2446080037566
3240000 153.57381140999874
3250000 161.7841366710623
3260000 163.559308296613
3270000 158.93603369931196
3280000 173.32314770892773
3290000 178.24024381991978
3300000 182.90832062059604
3310000 168.16867780241878
3320000 164.2355652116352
3330000 156.00593130446154
3340000 170.34073252382666
3350000 166.7871279848454
3360000 161.48045544203626
3370000 168.89890205868
3380000 157.2614482478688
3390000 154.41978471788156
3400000 160.9994548488381
3410000 168.45603955819044
3420000 167.57775572

6130000 159.40518179285945
6140000 160.9065479442088
6150000 154.28855529048667
6160000 146.91108140432095
6170000 154.39951404172433
6180000 155.24006306007468
6190000 158.7384533785656
6200000 158.01673189835958
6210000 159.3710467909343
6220000 155.7055190374916
6230000 173.4086127369815
6240000 161.47646947563334
6250000 169.1161181376878
6260000 159.79642532485283
6270000 160.01687747434335
6280000 177.42683590789377
6290000 159.55506233232734
6300000 177.8522611246379
6310000 158.43591632422186
6320000 158.79154548910796
6330000 147.88960440286928
6340000 156.49373727454605
6350000 151.75212726150923
6360000 156.52603743477476
6370000 152.44779354403983
6380000 149.44633864748425
6390000 148.524599810946
6400000 152.32772157926905
6410000 162.26996030864603
6420000 160.46736218679632
6430000 156.21915904965485
6440000 155.75086673067233
6450000 170.6372791257308
6460000 172.73460225593945
6470000 159.26192659286252
6480000 149.42519374282014
6490000 148.06163418000264
6500000 149

9210000 156.02612131357048
9220000 149.77545487738513
9230000 156.46289150367352
9240000 148.13536819092488
9250000 158.85941652854316
9260000 155.93546871652072
9270000 161.92517708467395
9280000 161.87676661322988
9290000 150.72581417912102
9300000 153.2339020695347
9310000 147.56112343258644
9320000 150.86310800617778
9330000 150.94420240711338
9340000 140.05209371089404
9350000 145.1313989073673
9360000 156.55044970438692
9370000 154.2608196420123
9380000 162.60519353695597
9390000 148.07964924354368
9400000 149.31272414333975
9410000 164.85342929853027
9420000 166.6526775970825
9430000 140.24248539519246
9440000 52.74684708809813
9450000 53.46544808327785
9460000 52.966664468507936
9470000 53.15641941194996
9480000 52.790970910003175
9490000 147.59380818136358
9500000 185.63838660788085
9510000 159.64425399590255
9520000 150.02853539081121
9530000 163.55240266639976
9540000 169.78558284909218
9550000 160.40888199638556
9560000 160.01170065332695
9570000 143.42509138187242
9580000 

12210000 154.50697014306562
12220000 149.61733214058145
12230000 150.14045361095395
12240000 139.27688924012418
12250000 151.21172461200922
12260000 149.09610256561646
12270000 159.1708680503219
12280000 152.5948182170409
12290000 161.50090469199932
12300000 158.70350071782323
12310000 152.1214751047638
12320000 159.12024669832016
12330000 151.90014905450516
12340000 149.99260940797015
12350000 139.3006376501131
12360000 148.38217407661574
12370000 141.22025235374096
12380000 148.86817564948998
12390000 141.55095319192304
12400000 143.9590679888197
12410000 151.73753809869854
12420000 146.96029843022677
12430000 140.74924072346514
12440000 134.57074174043316
12450000 142.42141871908674
12460000 132.2658966226368
12470000 150.31914523868966
12480000 164.4708728857456
12490000 71.51624548361718
12500000 22.29754983202217
12510000 21.379745715872836
12520000 21.33231777205064
12530000 21.301246158796886
12540000 35.28591823526184
12550000 181.08022650795732
12560000 165.59562297156526
125

...

237570000 3.958937616957062
237580000 3.8613926416057156
237590000 3.7939263906992013
237600000 3.91071821416531
237610000 3.9438083713671768
237620000 1.8549308081082967
237630000 1.8630666606236959
237640000 3.0328217824973467
237650000 4.06212710991776
237660000 4.075962982479202
237670000 3.695707207299411
237680000 4.03489942018056
237690000 4.428786614238037
237700000 4.537863981842379
237710000 4.034963262493643
237720000 4.426051350176838
237730000 4.645775839389802
237740000 2.4745022570480244
237750000 2.2451561636977724
237760000 2.44808365663596
237770000 4.33592644491897
237780000 3.917703692067021
237790000 3.5444485364220544
237800000 3.5604361583259836
237810000 3.8022330561928617
237820000 3.6702871706546545
237830000 1.2692899949545793
237840000 0.9150521810582081
237850000 1.339471157679995
237860000 4.014243658366296
237870000 3.6548295134757893
237880000 4.044769168827375
237890000 3.965987392568648
237900000 3.7488438969009783
237910000 3.916446225962222
237920000

240450000 3.3698441614002457
240460000 2.7136985571731134
240470000 3.0542972230563827
240480000 3.111166333904551
240490000 3.1605963629555105
240500000 3.471412739654335
240510000 0.9435963103623987
240520000 0.7572005242908597
240530000 0.9789514655780196
240540000 3.9020395002944075
240550000 3.0889043435382444
240560000 3.14008555217688
240570000 3.1679665887085395
240580000 2.845485371722281
240590000 3.2557070735384226
240600000 3.43468761227868
240610000 2.8573971244076493
240620000 2.8585792355631456
240630000 3.4060873375519978
240640000 3.567680142501473
240650000 1.6730924317558806
240660000 1.3124366905217104
240670000 1.4739126218591532
240680000 1.6019392532016832
240690000 3.525348364168121
240700000 3.3366068171396255
240710000 3.320476460431529
240720000 3.2879016625797615
240730000 3.183759626438081
240740000 3.291917528011507
240750000 3.7773663280663357
240760000 3.1401496915059686
240770000 3.1202134674428375
240780000 3.054162652113073
240790000 3.554653662872851

243150000 1.9867751383387247
243160000 1.8629260945475896
243170000 1.1067924960076612
243180000 1.849562820109122
243190000 2.112045833122233
243200000 2.0963430459774863
243210000 2.3927006818643743
243220000 1.9168363722880484
243230000 1.980149278859284
243240000 1.3239145790935622
243250000 1.2992797187685767
243260000 1.8386097843786013
243270000 1.9301939077341754
243280000 0.4438606692811847
243290000 1.5064452007379665
243300000 1.9063543416172994
243310000 1.6294357311387657
243320000 1.1779416481810012
243330000 1.902045512038542
243340000 1.7450070744339825
243350000 2.087376709457278
243360000 2.056389348704894
243370000 2.0421322647852698
243380000 2.0482643696920078
243390000 1.4590335520796773
243400000 0.8929656798201798
243410000 1.8889456169681351
243420000 1.5293085541716482
243430000 0.4928267597272992
243440000 1.5816882108551462
243450000 1.9777186210561395
243460000 1.1777048465157944
243470000 1.4005365271039538
243480000 1.813172758082496
243490000 1.919849084

245990000 1.0452234235428572
246000000 1.0297037686118815
246010000 0.9906483267250061
246020000 1.0411897343193293
246030000 1.0500526230498022
246040000 1.172501860391895
246050000 0.9979836961237921
246060000 0.8998609279060298
246070000 1.0054095003071826
246080000 0.9921649193750091
246090000 0.6844320702779293
246100000 0.7166125508467555
246110000 0.2438108593276607
246120000 0.6985150558849971
246130000 0.5976457255004645
246140000 1.0415410066350566
246150000 1.0101560505340894
246160000 0.8243414071768126
246170000 0.9882006723891099
246180000 1.0332596879388756
246190000 1.021470545492649
246200000 0.908040998337547
246210000 0.9068888144626286
246220000 1.0023940650753578
246230000 0.8297817782152295
246240000 0.5716201769269507
246250000 0.6656311012423038
246260000 0.22777555391046075
246270000 0.7863185599858894
246280000 0.4404727482328415
246290000 0.9650549776474503
246300000 0.9494231740170916
246310000 0.9492859558892449
246320000 0.9427345890351866
246330000 1.0295

248800000 0.15357730755227803
248810000 0.15220527317042484
248820000 0.15365580885033475
248830000 0.15826909395202002
248840000 0.14894492161858083
248850000 0.11610325791016553
248860000 0.05955986997077862
248870000 0.0762260622737209
248880000 0.1390670796267589
248890000 0.14194549009180069
248900000 0.14387542291084926
248910000 0.11995951146648991
248920000 0.11502532057493925
248930000 0.1026053361518979
248940000 0.029096686167418954
248950000 0.10665459455690779
248960000 0.11336251005807851
248970000 0.11661532484690347
248980000 0.11257278780192137
248990000 0.10035046385707458
249000000 0.08322437397423055
249010000 0.04718843701205651
249020000 0.07737555296558142
249030000 0.10165331145174636
249040000 0.09219798658198118
249050000 0.09545489154211018
249060000 0.09195121281940408
249070000 0.0875544743193388
249080000 0.05627873687464661
249090000 0.018730207340882883
249100000 0.07772491432694593
249110000 0.07215592278487153
249120000 0.07470080837796794
249130000 0.